In [1]:
import laser_lib
import numpy as np
import math
import time
import cv2
from skimage.color import rgb2lab, lab2rgb

In [2]:
queue = laser_lib.DacQueue()

Found  1 Helios DACs


In [3]:
def signer(f, T, phase=0.0):
    '''
    Produces spaced out points from 0 to 1 whose distance between 
    consecutive points is sinusoidal 
    Input:  f - frequency 
            T - number of samples
            phase - phase shift (0, 1)
    Returns: x - positions on the line
             dx - the derivative of the function of x
    '''
    # Convert to radians
    f = f*2*np.pi
    # Generate the baseline 
    t = np.linspace(0.0+phase, 1.0+phase, T)
    # Find points 
    x = (f*t - np.cos(f*t) + 1)/f
    # Calculate derivative
    dx = (np.sin(f*t) + 1)/2
    
    return x, dx

def triangle(theta, alpha, sincos='sin'):
    '''
    Returns a wave between a sine wave and triangle wave depending on alpha
    alpha = 0 - sine wave
    alpha = 1 - triangle wave
    '''
    if(sincos == 'sin'):
        ps = 0
    elif(sincos == 'cos'):
        ps = np.pi/2
    return alpha*2*np.arcsin(np.sin(theta + ps))/np.pi + (1-alpha)*np.sin(theta + ps)
    
def ellipser(scale_x=1, scale_y=1, f_x=1, f_y=1, f_s=1000, phase_s=0, sintriang=0, colors=[(1,1,1)], T=500):
    '''
    Generates a pattern that is a function of ellipses 
    with parameters that can be interpolated between
    '''
    theta_norm, theta_dx = signer(f=f_s, phase=phase_s, T=T)
    theta = theta_norm*2*np.pi
    arr_pos = np.zeros((T, 2))
    arr_pos[:,0] = scale_x*triangle(f_x*theta, sintriang, 'sin')
    arr_pos[:,1] = scale_y*triangle(f_y*theta, sintriang, 'cos')
    
    # Color the pattern
    arr_col = np.ones((T, 3), np.float32)
    if(len(colors) == 1):
        # All one color
        arr_col[:, :] = colors[0][np.newaxis, :]
    if(len(colors) == 2):
        # Color based on speed of laser
        arr_col[theta_dx > 0.1, :] = colors[0]
        arr_col[theta_dx <= 0.1, :] = colors[1]
    
    return arr_pos, arr_col


### Beam pairs flashing back and forth

In [4]:
T = 1000
theta = np.linspace(0, 2*np.pi, T)
freq = 10
arr_pos = np.zeros((T, 2))
arr_pos_l = np.zeros((T, 2))
arr_pos_r = np.zeros((T, 2))
arr_col_l = np.zeros((T, 3))
arr_col_r = np.zeros((T, 3))
scaler = 0.75
arr_pos[:, 0] = np.cos(freq*theta)
arr_pos_l[:, 0] = np.cos(freq*theta)*scaler + (1-scaler)
arr_pos_r[:, 0] = np.cos(freq*theta)*scaler - (1-scaler)
arr_col_l[arr_pos[:,0] >  0.99, :] = np.array([1,1,1])/6
arr_col_r[arr_pos[:,0] < -0.99, :] = np.array([1,1,1])/6
arr_col_l[arr_pos[:,0] < -0.99, :] = np.array([1,0,1])
arr_col_r[arr_pos[:,0] >  0.99, :] = np.array([1,0,1])

stay = 10
while(True):
    for i in range(stay):
        queue.submit(arr_pos_l, arr_col_l)
        
    for i in range(stay):
        queue.submit(arr_pos_r, arr_col_r)

KeyboardInterrupt: 

### Concentric, Rotating, Discrete Beam Circles

In [5]:
def rotate_pos(arr_pos, ang_rot):
    '''
    Rotates all points in arr_pos by ang_rot radians
    '''
    c, s = np.cos(ang_rot), np.sin(ang_rot)
    R = np.array(((c, -s), (s, c)))
    return np.matmul(arr_pos, R)

ang_rot = 0
T = 300
num_beams = 6
num_cols = 3
points_per_beam = T//num_beams
trip_col = np.array([[1,0,1],[0,0.5,1],[0,0,1]])
trip_col = np.tile(trip_col, (num_beams//num_cols, 1))
trip_col = np.repeat(trip_col, points_per_beam, axis=0)

while(True):
    ang_rot += 0.01
    arr_pos, arr_col = ellipser(scale_x=1, scale_y=1, f_x=1, f_y=1, f_s=num_beams, phase_s=0, sintriang=0, colors=[(0,0,0),(1,1,1)], T=T)
    queue.submit(rotate_pos(arr_pos, ang_rot), trip_col*arr_col)
    queue.submit(0.75*rotate_pos(arr_pos, -ang_rot), trip_col*arr_col)
    queue.submit(0.50*rotate_pos(arr_pos,  ang_rot), trip_col*arr_col)
    

KeyboardInterrupt: 

In [ ]:
# Frames -> Patterns -> Sequences

In [6]:
# ARC THAT EXPANDS
T = 500
theta = np.concatenate([np.linspace(0, np.pi, T//2), np.linspace(np.pi, 0, T//2)])
arr_pos = np.zeros((T, 2))
arr_pos[:, 0] = np.cos(theta)
arr_pos[:, 1] = np.sin(theta)

arr_col = np.ones((T, 3))

num_steps = 100
while(True):
    #Arc disappears from right and left side of circle (0)     
    for i in range(num_steps//2):
        arr_col_out = arr_col*np.logical_and(theta[:, np.newaxis] > np.pi*i/num_steps, theta[:, np.newaxis] < np.pi - np.pi*i/num_steps)
        queue.submit(arr_pos, arr_col_out)
        
        


KeyboardInterrupt: 

In [ ]:
np.zeros((5, 3))[...,np.newaxis].shape

## Straight Up Villain (Chorus) 

In [ ]:
# This pattern is for the chorus, it is repeated throughout the song

# Time (seconds) between beats
Tb = 0.375
# Number of frames per beat
T = int(Tb*queue.dac_rate)

# 1) Beam fast move twice (2 beats)
# 2) Beam slow move once (2 beats)
# 3) Beams split

# test with just one beam 
p1 = np.array([-1, -1])
p2 = np.array([ 1, -1])

arr_pos = np.linspace(p1, p2, T)
arr_col = np.ones((T, 3))
print(arr_pos.shape, arr_col.shape)
queue.submit(arr_pos, arr_col)
    


In [ ]:
# Start off with a certain number of beams 
num_beams_init = 2
num_splits = 3
num_beams_end = num_beams_init**(num_splits+1)

axes = 1 # Symmetries, this increases w/ every split


In [ ]:
num_beams_end

In [7]:
# ARC THAT EXPANDS and collapes formulas
T = 1000
theta = np.concatenate([np.linspace(0, np.pi, T//2), np.linspace(np.pi, 0, T//2)])
arr_pos = np.zeros((T, 2))
arr_pos[:, 0] = np.cos(theta)
arr_pos[:, 1] = np.sin(theta)

arr_col = np.ones((T, 3))

num_steps = 100
#Arc expands in both directions from Top of circle (pi/2)
for i in range(num_steps//2):
    arr_col_out = arr_col*np.logical_and(theta[:, np.newaxis] < np.pi*i/num_steps + np.pi/2, theta[:, np.newaxis] > np.pi/2 - np.pi*i/num_steps)
    queue.submit(arr_pos, arr_col_out)

#Arc disappears in both directions from Top of circle (pi/2)    
for i in range(num_steps//2):
    arr_col_out = arr_col*np.logical_or(theta[:, np.newaxis] > np.pi*i/num_steps + np.pi/2, theta[:, np.newaxis] < np.pi/2 - np.pi*i/num_steps)
    queue.submit(arr_pos, arr_col_out)

#Arc disappears in both directions from 45deg of circle (pi/4)     
for i in range(num_steps//2):
    arr_col_out = arr_col*np.logical_or(theta[:, np.newaxis] > np.pi*i/num_steps + np.pi/4, theta[:, np.newaxis] < np.pi/4 - np.pi*i/num_steps)
    queue.submit(arr_pos, arr_col_out)
    
#Arc disappears in both directions from 135deg of circle (3pi/4)     
for i in range(num_steps//2):
    arr_col_out = arr_col*np.logical_or(theta[:, np.newaxis] > np.pi*i/num_steps + 3*np.pi/4, theta[:, np.newaxis] < 3*np.pi/4 - np.pi*i/num_steps)
    queue.submit(arr_pos, arr_col_out)
    
#Arc disappears from right and left side of circle (0 and pi)     
for i in range(num_steps//2):
    arr_col_out = arr_col*np.logical_and(theta[:, np.newaxis] > np.pi*i/num_steps, theta[:, np.newaxis] < np.pi - np.pi*i/num_steps)
    queue.submit(arr_pos, arr_col_out)

In [ ]:
#Expanding and disappearing arc v1
T = 1000
theta = np.concatenate([np.linspace(0, np.pi, T//2), np.linspace(np.pi, 0, T//2)])
arr_pos = np.zeros((T, 2))
arr_pos[:, 0] = np.cos(theta)
arr_pos[:, 1] = np.sin(theta)

arr_col = np.ones((T, 3))
finger_size = 0.025

def inside(arr_pos, pos, width):
    return np.logical_and(arr_pos[:, np.newaxis] > (pos-width), arr_pos[:, np.newaxis] < (pos+width))

print(np.pi/2, theta)
#num steps = 30fps * 6 seconds + 1 beat, 1 beat = 3/8s = 90/8 frams= 11.25 or so frames, quick quick slow = 11, 11, 23 = 45 = 11.25*4
num_steps = 191
#Arc expands in both directions from Top of circle (pi/2)
for i in range(num_steps):
    # Draw the points (they appear sequentially)
    arr_col_out = arr_col*inside(theta, np.pi/2, finger_size)
    arr_col_out += arr_col*inside(theta, np.pi/4, finger_size)*(i > 5)
    arr_col_out += arr_col*inside(theta, np.pi*3/4, finger_size)*(i > 5)
    arr_col_out += arr_col*inside(theta, 0, finger_size)*(i > 10)
    arr_col_out += arr_col*inside(theta, np.pi, finger_size)*(i > 10)
    if(i > 11):
        arr_col_out = 0
        i_step = i-11
        arr_col_out += arr_col*np.logical_and(theta[:, np.newaxis] < np.pi*i_step/90 + np.pi/2, theta[:, np.newaxis] > np.pi/2 - np.pi*i_step/90)
    #Add "Snatch" should finish at i=56 (1 beat + 1.5s) - is this possible? Laser not running from pi to 0 in line, only in circle. Does path need to include diameter of semi circle?
    #if(i>45):
        #i_step = i-45
        #arr_col_out = i_step/11 
    #if(i>56):
        #arr_col_out = 0
    #Arc expands from center
    #i -= 90
    # Delete the arc
    if(i > 101):
        i_step = i-101
        arr_col_out = arr_col*np.logical_or(theta[:, np.newaxis] > np.pi*i_step/90 + np.pi/2, theta[:, np.newaxis] < np.pi/2 - np.pi*i_step/90)
        arr_col_out += arr_col*inside(theta, np.pi/2, finger_size)
    ##i -= 17
    #Arc starts deleting from pi/4 and 3pi/4 as well. Also add point at pi/4, and 3pi/4
    if(i > 118):
        arr_col_out += arr_col*np.logical_or(theta[:, np.newaxis] > np.pi*i_step/90 + np.pi/4, theta[:, np.newaxis] < np.pi/4 - np.pi*i_step/90)
        arr_col_out += arr_col*np.logical_or(theta[:, np.newaxis] > np.pi*i_step/90 + 3*np.pi/4, theta[:, np.newaxis] < 3*np.pi/4 - np.pi*i_step/90)
        arr_col_out += arr_col*inside(theta, np.pi/4, finger_size)
        arr_col_out += arr_col*inside(theta, np.pi*3/4, finger_size)
        #i -= 17
    #Arc starts deleting from 0 and pi as well. Also add point at 0, and pi/4
    if(i > 135)
        arr_col_out = arr_col*np.logical_and(theta[:, np.newaxis] > np.pi*i/90, theta[:, np.newaxis] < np.pi - np.pi*i/90)
        arr_col_out += arr_col*inside(theta, 0, finger_size)
        arr_col_out += arr_col*inside(theta, np.pi, finger_size)
    #Add "Snatch" should finish at i=146 (3s after first) - is this possible? Laser not running from pi to 0 in line, only in circle. Does path need to include diameter of semi circle?
        #i_step = i-135
        #arr_col_out = i_step/11 
    #if(i>146):
        #arr_col_out = 0? don't want all 0, just the snatch
    
    queue.submit(arr_pos, arr_col_out)


In [ ]:
# Opening Beats 0-8 seconds, switching rays
T = 1000
theta = np.concatenate([np.linspace(0, np.pi, T//2), np.linspace(np.pi, 0, T//2)])
arr_pos = np.zeros((T, 2))
arr_pos[:, 0] = np.cos(theta)
arr_pos[:, 1] = np.sin(theta)

arr_col = np.ones((T, 3))

#not eliminating lasers right now. Just have timing down, do i need to manually clear after each line? more efficient code?
num_steps = 330
for i in range(num_steps):
    #0-1.5s
    if (i>11): #.375s (a)
        arr_col_out += arr_col*inside(theta, np.pi*(2/3), finger_size)
        arr_col_out += arr_col*inside(theta, np.pi*/6, finger_size)
        arr_col_out = 0
    if (i>22): #.75s (b)
        arr_col_out += arr_col*inside(theta, np.pi*/3, finger_size)
        arr_col_out += arr_col*inside(theta, np.pi*(5/6), finger_size)
        arr_col_out = 0
    if (i>33): #1.125s (a)
        arr_col_out += arr_col*inside(theta, np.pi*(2/3), finger_size)
        arr_col_out += arr_col*inside(theta, np.pi*/6, finger_size)
        arr_col_out = 0
    if (i>45): #1.5s (b correction)
        arr_col_out += arr_col*inside(theta, np.pi*/3, finger_size)
        arr_col_out += arr_col*inside(theta, np.pi*(5/6), finger_size)
        arr_col_out = 0
    
    #1.5-3s
    if (i>56): #1.875s (a)
        arr_col_out += arr_col*inside(theta, np.pi*(2/3), finger_size)
        arr_col_out += arr_col*inside(theta, np.pi*/6, finger_size)
        arr_col_out = 0
    if (i>67): #2.25s (b)
        arr_col_out += arr_col*inside(theta, np.pi*/3, finger_size)
        arr_col_out += arr_col*inside(theta, np.pi*(5/6), finger_size)
        arr_col_out = 0
    if (i>78): #2.625s (a)
        arr_col_out += arr_col*inside(theta, np.pi*(2/3), finger_size)
        arr_col_out += arr_col*inside(theta, np.pi*/6, finger_size)
        arr_col_out = 0
    if (i>90): #3s (b correction)
        arr_col_out += arr_col*inside(theta, np.pi*/3, finger_size)
        arr_col_out += arr_col*inside(theta, np.pi*(5/6), finger_size)
        arr_col_out = 0

    #3s-4.5s
    if (i>101): #3.375s (a + 1)
        arr_col_out += arr_col*inside(theta, np.pi*(2/3), finger_size)
        arr_col_out += arr_col*inside(theta, np.pi*/6, finger_size)
        arr_col_out = arr_col*inside(theta, np.pi/2, finger_size)
        arr_col_out = 0
    if (i>112): #3.75s (b)
        arr_col_out += arr_col*inside(theta, np.pi*/3, finger_size)
        arr_col_out += arr_col*inside(theta, np.pi*(5/6), finger_size)
        arr_col_out = 0
    if (i>117): #3.9375s (2)
        arr_col_out += arr_col*inside(theta, np.pi*/4, finger_size)
        arr_col_out += arr_col*inside(theta, np.pi*(3/4), finger_size)
        arr_col_out = 0
    if (i>123): #4.125s (a)
        arr_col_out += arr_col*inside(theta, np.pi*(2/3), finger_size)
        arr_col_out += arr_col*inside(theta, np.pi*/6, finger_size)
        arr_col_out = 0
    if (i>135): #4.5s (b + 3 correction)
        arr_col_out += arr_col*inside(theta, np.pi*/3, finger_size)
        arr_col_out += arr_col*inside(theta, np.pi*(5/6), finger_size)
        arr_col_out += arr_col*inside(theta, 0, finger_size)
        arr_col_out += arr_col*inside(theta, np.pi, finger_size)
        arr_col_out = 0
    #4.5s-6s
    if (i>146): #4.875s (a)
        arr_col_out += arr_col*inside(theta, np.pi*(2/3), finger_size)
        arr_col_out += arr_col*inside(theta, np.pi*/6, finger_size)
        arr_col_out = 0
    if (i>157): #5.25s (b)
        arr_col_out += arr_col*inside(theta, np.pi*/3, finger_size)
        arr_col_out += arr_col*inside(theta, np.pi*(5/6), finger_size)
        arr_col_out = 0
    if (i>168): #5.625s (a)
        arr_col_out += arr_col*inside(theta, np.pi*(2/3), finger_size)
        arr_col_out += arr_col*inside(theta, np.pi*/6, finger_size)
        arr_col_out = 0
    if (i>180): #6s (b correction)
        arr_col_out += arr_col*inside(theta, np.pi*/3, finger_size)
        arr_col_out += arr_col*inside(theta, np.pi*(5/6), finger_size)
        arr_col_out = 0
        arr_col_out = arr_col*inside(theta, np.pi/2, finger_size)
    #6s-7.5s    
    if (i>185): #6.1875s (1+2)
        arr_col_out += arr_col*inside(theta, np.pi*/4, finger_size)
        arr_col_out += arr_col*inside(theta, np.pi*(3/4), finger_size)
    if (i>191): #6.375s (a + 1+2+3 correction)
        arr_col_out += arr_col*inside(theta, np.pi*(2/3), finger_size)
        arr_col_out += arr_col*inside(theta, np.pi*/6, finger_size)
        arr_col_out += arr_col*inside(theta, 0, finger_size)
        arr_col_out += arr_col*inside(theta, np.pi, finger_size)
        arr_col_out = 0
    if (i>202): #6.75s (b)
        arr_col_out += arr_col*inside(theta, np.pi*/3, finger_size)
        arr_col_out += arr_col*inside(theta, np.pi*(5/6), finger_size)
        arr_col_out = 0
    if (i>213): #7.125s (a)
        arr_col_out += arr_col*inside(theta, np.pi*(2/3), finger_size)
        arr_col_out += arr_col*inside(theta, np.pi*/6, finger_size)
        arr_col_out = 0
    if (i>225): #7.5s (b correction)
        arr_col_out += arr_col*inside(theta, np.pi*/3, finger_size)
        arr_col_out += arr_col*inside(theta, np.pi*(5/6), finger_size)
        arr_col_out = 0
    #7.5-9s
    if (i>236): #7.875s (a)
        arr_col_out += arr_col*inside(theta, np.pi*(2/3), finger_size)
        arr_col_out += arr_col*inside(theta, np.pi*/6, finger_size)
        arr_col_out = 0
    if (i>247): #8.25s (b)
        arr_col_out += arr_col*inside(theta, np.pi*/3, finger_size)
        arr_col_out += arr_col*inside(theta, np.pi*(5/6), finger_size)
        arr_col_out = 0
    if (i>252): #8.4375s (b)
        arr_col_out += arr_col*inside(theta, np.pi*/3, finger_size)
        arr_col_out += arr_col*inside(theta, np.pi*(5/6), finger_size)
        arr_col_out = 0
    if (i>258): #8.625s (a)
        arr_col_out += arr_col*inside(theta, np.pi*(2/3), finger_size)
        arr_col_out += arr_col*inside(theta, np.pi*/6, finger_size)
        arr_col_out = 0
    if (i>264): #8.8125s (a correction)
        arr_col_out += arr_col*inside(theta, np.pi*(2/3), finger_size)
        arr_col_out += arr_col*inside(theta, np.pi*/6, finger_size)
        arr_col_out = 0
    if (i>270): #9s (b)
        arr_col_out += arr_col*inside(theta, np.pi*/3, finger_size)
        arr_col_out += arr_col*inside(theta, np.pi*(5/6), finger_size)
        arr_col_out = 0
    #9s-11s
    if (i>275): #9.1875s (b)
        arr_col_out += arr_col*inside(theta, np.pi*/3, finger_size)
        arr_col_out += arr_col*inside(theta, np.pi*(5/6), finger_size)
        arr_col_out = 0
    #9.375 Ripple How to? function based on i that increases distance travelled as i increases like expanding circles. 

        
    i-= 11 #4.875s (a)
    arr_col_out = arr_col*np.logical_and(theta[:, np.newaxis] = 2*np.pi/3, theta[:, np.newaxis] = np.pi/6)
    i-= 11 #5.25s (b)
    arr_col_out = arr_col*np.logical_and(theta[:, np.newaxis] = 5*np.pi/6, theta[:, np.newaxis] = np.pi/3)
    i-= 11 #5.625s (a)
    arr_col_out = arr_col*np.logical_and(theta[:, np.newaxis] = 2*np.pi/3, theta[:, np.newaxis] = np.pi/6)
    i-= 12 #6s (correction) (b) (1)
    arr_col_out = arr_col*np.logical_and(theta[:, np.newaxis] = 5*np.pi/6, theta[:, np.newaxis] = np.pi/3, theta[:, np.newaxis] = np.pi/2)
    i-= 5  #6.1875s (1) (2)
    arr_col_out = arr_col*np.locgical_and(theta[:, np.newaxis] = np.pi/4, theta[:, np.newaxis] = 3*np.pi/4)
    i-= 6  #6.375s (a) (1) (2) (3)
    arr_col_out = arr_col*np.logical_and(theta[:, np.newaxis] = 2*np.pi/3, theta[:, np.newaxis] = np.pi/6, theta[:, np.newaxis] = 0, theta[:, np.newaxis] = np.pi)
    i-= 11 #6.75s (b)
    arr_col_out = arr_col*np.logical_and(theta[:, np.newaxis] = 5*np.pi/6, theta[:, np.newaxis] = np.pi/3)
    i-= 11 #7.125s (a)
    arr_col_out = arr_col*np.logical_and(theta[:, np.newaxis] = 2*np.pi/3, theta[:, np.newaxis] = np.pi/6)
    i-= 12 #7.5s (correction) (b)
    arr_col_out = arr_col*np.logical_and(theta[:, np.newaxis] = 5*np.pi/6, theta[:, np.newaxis] = np.pi/3)
    i-= 11 #7.875s (a)
    arr_col_out = arr_col*np.logical_and(theta[:, np.newaxis] = 2*np.pi/3, theta[:, np.newaxis] = np.pi/6)
    i-= 11 #8.25s (b)
    arr_col_out = arr_col*np.logical_and(theta[:, np.newaxis] = 5*np.pi/6, theta[:, np.newaxis] = np.pi/3)
    i-= 5  #8.4375s (b)
    arr_col_out = arr_col*np.logical_and(theta[:, np.newaxis] = 5*np.pi/6, theta[:, np.newaxis] = np.pi/3)
    i-= 6  #8.625s (a)
    arr_col_out = arr_col*np.logical_and(theta[:, np.newaxis] = 2*np.pi/3, theta[:, np.newaxis] = np.pi/6)
    i-= 6  #8.8125s (correction) (a)
    arr_col_out = arr_col*np.logical_and(theta[:, np.newaxis] = 2*np.pi/3, theta[:, np.newaxis] = np.pi/6)
    i-= 6  #9s (b)
    arr_col_out = arr_col*np.logical_and(theta[:, np.newaxis] = 5*np.pi/6, theta[:, np.newaxis] = np.pi/3)
    i-= 5 #9.1875 (b)
    arr_col_out = arr_col*np.logical_and(theta[:, np.newaxis] = 5*np.pi/6, theta[:, np.newaxis] = np.pi/3)
    i-= 6 #9.375 (ripple)

In [ ]:
# ARC THAT EXPANDS and collapses in order
T = 1000
theta = np.concatenate([np.linspace(0, np.pi, T//2), np.linspace(np.pi, 0, T//2)])
arr_pos = np.zeros((T, 2))
arr_pos[:, 0] = np.cos(theta)
arr_pos[:, 1] = np.sin(theta)

arr_col = np.ones((T, 3))

#num steps = 30fps * 6 seconds + 1 beat, 1 beat = 3/8s = 90/8 frams= 11.25 or so frames, quick quick slow = 11, 11, 23 = 45 = 11.25*4
num_steps = 191
#Arc expands in both directions from Top of circle (pi/2)
for i in range(num_steps):
    arr_col_out = arr_col*(theta[:, np.newaxis] == np.pi/2)
    i-= 5
    arr_col_out = arr_col*np.logical_and(theta[:, np.newaxis] == np.pi/4, theta[:, np.newaxis] == 3*np.pi/4)
    i-= 5
    arr_col_out = arr_col*np.logical_and(theta[:, np.newaxis] == 0, theta[:, np.newaxis] == np.pi)
    i-= 1
    arr_col_out = arr_col*np.logical_and(theta[:, np.newaxis] < np.pi*i/num_steps + np.pi/2, theta[:, np.newaxis] > np.pi/2 - np.pi*i/num_steps)
    i -= 90
    arr_col_out1 = arr_col*np.logical_or(theta[:, np.newaxis] > np.pi*i/num_steps + np.pi/2, theta[:, np.newaxis] < np.pi/2 - np.pi*i/num_steps)
    arr_col_out2 = arr_col*(theta[:, np.newaxis] == np.pi/2)
    arr_col_out = np.logical_and(arr_col_out1, arr_col_out2)
    i -= 17 
    arr_col_out1 = arr_col*np.logical_or(theta[:, np.newaxis] > np.pi*i/num_steps + np.pi/4, theta[:, np.newaxis] < np.pi/4 - np.pi*i/num_steps)
    arr_col_out2 = arr_col*np.logical_or(theta[:, np.newaxis] > np.pi*i/num_steps + 3*np.pi/4, theta[:, np.newaxis] < 3*np.pi/4 - np.pi*i/num_steps)
    arr_col_out3 = arr_col*np.logical_and(theta[:, np.newaxis] == np.pi/4, theta[:, np.newaxis] == 3*np.pi/4)
    arr_col_out = np.logical_and(arr_col_out1, arr_col_out2, arr_col_out3)
    i -= 17
    arr_col_out1 = arr_col*np.logical_and(theta[:, np.newaxis] > np.pi*i/num_steps, theta[:, np.newaxis] < np.pi - np.pi*i/num_steps)
    arr_col_out2 = arr_col*np.logical_and(theta[:, np.newaxis] == 0, theta[:, np.newaxis] == np.pi)
    arr_col_out = np.logical_and(arr_col_out1, arr_col_out2)
    
    queue.submit(arr_pos, arr_col_out)
#big concern is I don't think the individual rays will be gone. I think i will have them persist throughout, 
#but I need to reset arr.col.out to 0 for those points only. Write additional code to say such?

In [ ]:
# Opening Beats 0-8 seconds, switching rays
T = 1000
theta = np.concatenate([np.linspace(0, np.pi, T//2), np.linspace(np.pi, 0, T//2)])
arr_pos = np.zeros((T, 2))
arr_pos[:, 0] = np.cos(theta)
arr_pos[:, 1] = np.sin(theta)

arr_col = np.ones((T, 3))

#not eliminating lasers right now. Just have timing down, need to set up way to clear arr_col_out after each pass
num_steps = 330
for i in range(num_steps):
    i-= 11 #.375s (a)
    arr_col_out = arr_col*np.logical_and(theta[:, np.newaxis] = 2*np.pi/3, theta[:, np.newaxis] = np.pi/6)
    i-= 11 #.75s (b)
    arr_col_out = arr_col*np.logical_and(theta[:, np.newaxis] = 5*np.pi/6, theta[:, np.newaxis] = np.pi/3)
    i-= 11 #1.125s (a)
    arr_col_out = arr_col*np.logical_and(theta[:, np.newaxis] = 2*np.pi/3, theta[:, np.newaxis] = np.pi/6)
    i-= 12 #1.5s (correction) (b)
    arr_col_out = arr_col*np.logical_and(theta[:, np.newaxis] = 5*np.pi/6, theta[:, np.newaxis] = np.pi/3)
    i-= 11 #1.875s (a)
    arr_col_out = arr_col*np.logical_and(theta[:, np.newaxis] = 2*np.pi/3, theta[:, np.newaxis] = np.pi/6)
    i-= 11 #2.25s (b)
    arr_col_out = arr_col*np.logical_and(theta[:, np.newaxis] = 5*np.pi/6, theta[:, np.newaxis] = np.pi/3)
    i-= 11 #2.625s (a)
    arr_col_out = arr_col*np.logical_and(theta[:, np.newaxis] = 2*np.pi/3, theta[:, np.newaxis] = np.pi/6)
    i-= 12 #3s (correction) (b)
    arr_col_out = arr_col*np.logical_and(theta[:, np.newaxis] = 5*np.pi/6, theta[:, np.newaxis] = np.pi/3)
    i-= 11 #3.375s - First Background beat - (a) (1)
    arr_col_out = arr_col*np.logical_and(theta[:, np.newaxis] = 2*np.pi/3, theta[:, np.newaxis] = np.pi/6, theta[:, np.newaxis] = np.pi/2)
    i-= 11 #3.75s (b)
    arr_col_out = arr_col*np.logical_and(theta[:, np.newaxis] = 5*np.pi/6, theta[:, np.newaxis] = np.pi/3)
    i-= 6  #3.9375s (2)
    arr_col_out = arr_col*np.locgical_and(theta[:, np.newaxis] = np.pi/4, theta[:, np.newaxis] = 3*np.pi/4)
    i-= 5  #4.125 (a)
    arr_col_out = arr_col*np.logical_and(theta[:, np.newaxis] = 2*np.pi/3, theta[:, np.newaxis] = np.pi/6)
    i-= 12 #4.5 (correction) (b) (3)
    arr_col_out = arr_col*np.logical_and(theta[:, np.newaxis] = 5*np.pi/6, theta[:, np.newaxis] = np.pi/3, theta[:, np.newaxis] = 0, theta[:, np.newaxis] = np.pi)    
    i-= 11 #4.875s (a)
    arr_col_out = arr_col*np.logical_and(theta[:, np.newaxis] = 2*np.pi/3, theta[:, np.newaxis] = np.pi/6)
    i-= 11 #5.25s (b)
    arr_col_out = arr_col*np.logical_and(theta[:, np.newaxis] = 5*np.pi/6, theta[:, np.newaxis] = np.pi/3)
    i-= 11 #5.625s (a)
    arr_col_out = arr_col*np.logical_and(theta[:, np.newaxis] = 2*np.pi/3, theta[:, np.newaxis] = np.pi/6)
    i-= 12 #6s (correction) (b) (1)
    arr_col_out = arr_col*np.logical_and(theta[:, np.newaxis] = 5*np.pi/6, theta[:, np.newaxis] = np.pi/3, theta[:, np.newaxis] = np.pi/2)
    i-= 5  #6.1875s (1) (2)
    arr_col_out = arr_col*np.locgical_and(theta[:, np.newaxis] = np.pi/4, theta[:, np.newaxis] = 3*np.pi/4)
    i-= 6  #6.375s (a) (1) (2) (3)
    arr_col_out = arr_col*np.logical_and(theta[:, np.newaxis] = 2*np.pi/3, theta[:, np.newaxis] = np.pi/6, theta[:, np.newaxis] = 0, theta[:, np.newaxis] = np.pi)
    i-= 11 #6.75s (b)
    arr_col_out = arr_col*np.logical_and(theta[:, np.newaxis] = 5*np.pi/6, theta[:, np.newaxis] = np.pi/3)
    i-= 11 #7.125s (a)
    arr_col_out = arr_col*np.logical_and(theta[:, np.newaxis] = 2*np.pi/3, theta[:, np.newaxis] = np.pi/6)
    i-= 12 #7.5s (correction) (b)
    arr_col_out = arr_col*np.logical_and(theta[:, np.newaxis] = 5*np.pi/6, theta[:, np.newaxis] = np.pi/3)
    i-= 11 #7.875s (a)
    arr_col_out = arr_col*np.logical_and(theta[:, np.newaxis] = 2*np.pi/3, theta[:, np.newaxis] = np.pi/6)
    i-= 11 #8.25s (b)
    arr_col_out = arr_col*np.logical_and(theta[:, np.newaxis] = 5*np.pi/6, theta[:, np.newaxis] = np.pi/3)
    i-= 5  #8.4375s (b)
    arr_col_out = arr_col*np.logical_and(theta[:, np.newaxis] = 5*np.pi/6, theta[:, np.newaxis] = np.pi/3)
    i-= 6  #8.625s (a)
    arr_col_out = arr_col*np.logical_and(theta[:, np.newaxis] = 2*np.pi/3, theta[:, np.newaxis] = np.pi/6)
    i-= 6  #8.8125s (correction) (a)
    arr_col_out = arr_col*np.logical_and(theta[:, np.newaxis] = 2*np.pi/3, theta[:, np.newaxis] = np.pi/6)
    i-= 6  #9s (b)
    arr_col_out = arr_col*np.logical_and(theta[:, np.newaxis] = 5*np.pi/6, theta[:, np.newaxis] = np.pi/3)
    i-= 5 #9.1875 (b)
    arr_col_out = arr_col*np.logical_and(theta[:, np.newaxis] = 5*np.pi/6, theta[:, np.newaxis] = np.pi/3)
    i-= 6 #9.375 (ripple)

In [ ]:
# ARC THAT EXPANDS
T = 500
theta = np.concatenate([np.linspace(0, np.pi, T//2), np.linspace(np.pi, 0, T//2)])
arr_pos = np.zeros((T, 2))
arr_pos[:, 0] = np.cos(theta)
arr_pos[:, 1] = np.sin(theta)

arr_col = np.ones((T, 3))

num_steps = 100
while(True):
    for i in range(num_steps-30):
        arr_col_out = arr_col*np.logical_and(theta[:, np.newaxis] < np.pi*i/num_steps + np.pi/2, theta[:, np.newaxis] > np.pi/2 - np.pi*i/num_steps)
        i -= 20
        arr_col_out = arr_col_out*np.logical_or(theta[:, np.newaxis] > np.pi*i/num_steps + np.pi/2, theta[:, np.newaxis] < np.pi/2 - np.pi*i/num_steps)
        queue.submit(arr_pos, arr_col_out)

    for i in range(num_steps-30):
        arr_col_out = arr_col*np.logical_and(theta[:, np.newaxis] < np.pi*i/num_steps + np.pi/2, theta[:, np.newaxis] > np.pi/2 - np.pi*i/num_steps)
        i -= 20
        arr_col_out = arr_col_out*np.logical_or(theta[:, np.newaxis] > np.pi*i/num_steps + np.pi/2, theta[:, np.newaxis] < np.pi/2 - np.pi*i/num_steps)
        queue.submit(arr_pos, arr_col_out)



In [ ]:
b = x == 5
b

In [ ]:
x = np.linspace(0, 1, 8)
x
x == 1/2